# Cyber Grabs CTF 0x03 – Unbr34k4bl3

Credits: https://sekai.team/blog/cyber-grabs-ctf-0x03/unbr34k4bl3/

## Problem



```
No one can break my rsa encryption, prove me wrong !!

Flag Format: cybergrabs{}

Author: Mritunjya
```

In attachment 2 files:

source.py

```python
from Crypto.Util.number import *
from secret import *


assert (x>2 and x%2 == 0)
assert (isPrime(e1) and isPrime(e2))

def functor():
	val1 , val2 = 0,0
	for i in range(x+1):
		val1 += pow(e1,i)
	for j in range(3):
		val2 += pow(e2,j)
	assert (val1 == val2)

def keygen():
	while True:
		p,q = [getStrongPrime(1024) for _ in range(2)]
		if p%4==3 and q%4==3:
			break

	r = 2
	while True:
		r = r*x
		if r.bit_length()>1024 and isPrime(r-1):
			r = r-1
			break

	return p,q,r


functor()
p,q,r = keygen()
n = p*q*r
print(f"p:{p}")
print(f"q:{q}")
ip = inverse(p,q)
iq = inverse(q,p)
c1 = pow(bytes_to_long(flag[0:len(flag)//2].encode('utf-8')),e1,n)
c2 = pow(bytes_to_long(flag[len(flag)//2:].encode('utf-8')),e2,n)
print(f"n:{n}",f"ip:{ip}",f"iq:{iq}",f"c1:{c1}",f"c2:{c2}",sep="\n")
```

output.txt

In [1]:
n=267362205744654830055585746250317245125479735269853713372687604676608285629127977574310510441358104169652444917329986129098240750401425257601282268733834091593200445244725460613298199140690597119199763970064359847666802255456013592631532853951273286284878230893809080250386646832110506402289378691079462364884899662707502858007857457806853302449695351229004051902617728418480990341155900565542195318206284041182555579388392863474548687784403795738945489219689610881075059037192656116884269582257788959555951074322245033492165406470004019896763472332962300128378758934128374039937693688718317737657946435827745981009467876838127075176808098467305627394472135213533754815713468369763665632168616054982745256773112537152292099369137072982289095951236065885648588670059655452986720063260146952425798150221407866669449837430999779776718047668562687216933053536759554900663226163021145439386115076821161003965334731127329486856711654741683760749336235855319144478194501034662638054193682000283319917096796971
ip=65491313526527942082900846848440586365393305192439699810712229312474732937502934334921061033822729150056656630858908294464249602368303871630644420585085642204592189073314730233318796675949142968346807766087775542461078648703191450221286915401606901781524237580646760734493950360267230729125514156671619347616
iq=97034409222811998555255396847918439343239825222504093225438959283117395075159811973044380473862026342866489725039905931430797650466599952795602909181290621103197493223080488468216279214006070950393096075839913101687588555346523517436421698916141195686143520143972735534402754157166545851899187305574703394138
c1=103687839591259628532585171241634220321003599759860095236990117623065664975385083122971507015385215246948744078816596026772744294701233346732383214113445480056584639282712898073542520168025667980980057512174927564196375256682206601425714094930670415979638437119896258396784978194294581076901000507291277729888015413204446158926865037965291316577726275211006619643531704449499845352147547986667837681877488120093302675775792115380914560935989896453159186176952126083066619414338359303033325593504442257083571002878083287293828310810483726711816109297046925744157605591270761804522735216774801135342322479770391505911100485259078064775709124730966391629468398187269096529671187877954443617005248499140455160589093379715757808387108825458007733207099871941497372539249357162437077379731766825184301649010270921003130776410066972952756983157217280397531412843118202051922048479332111760976091302376602674590153876045380552746826056547929265785960676415919260117136285580971488670143947003566230254837742519
c2=171159809874438596904787534111610260851529969068192878049771299710688449419966698428704180474774734112617652498954998301185232279153644173070897800123538474930545720934844727376637921072749901149514789723141795042182408704214998390482343965532559149095934231081729041402598776401575561653660624208366051273601230345754361771067242657825194926706328336322383296953817730346429591680463526267530372572332663327157636745578067246913529155120642276894180354494816411827468256127607558873938451944866168777913756913920336763454881108023708284527878322162463081091624350220308273550298342755582044860337692076513609120342318151660103532559583052954725303030103413034880155621982581677423267299780543045375467310718078800411397780269409147558121862038983169509828944551199620508493589091401498720419409158373805529997911655270528589050795214164221299581104149954423726171539700223299445034347915430838395255700425648686205603925507474877720680274914513203566997846945579395522000899007446797091893230195801607


## Solving

Looking at the source code there are some important notes:

- $e_1$ and $e_2$ are related through $x$ secret (whom is even number and greater than $2$)
- $n=pqr$ is RSA multiprime using all prime factor
- $p,q \equiv 3 \bmod 4$. This suggests that the cryptosystem is actually a [Rabin cryptosystem](https://en.wikipedia.org/wiki/Rabin_cryptosystem).


### Finding $e_1$ and $e_2$

Looking at functor function:

```python
assert (x>2 and x%2 == 0)
assert (isPrime(e1) and isPrime(e2))

def functor():
	val1 , val2 = 0,0
	for i in range(x+1):
		val1 += pow(e1,i)
	for j in range(3):
		val2 += pow(e2,j)
	assert (val1 == val2)
```

It is possible express it as follow:

$$e_2^2 + e_2 + 1 = \sum_{i=0}^{x=2n}{e_1^i}$$

This expression must be assert to use defined cryptographic system, so $e_1$ and $e_2$ are related through $x=2n \gt 2$. In accorting to assertation some condition must be met:

- $e_1 \ll e_2$
- $x$ not so much big

Applying $\mod e_1$ to above equation:

$e_2^2 + e_2 + 1 \mod e_1 = 1 + \sum_{i=1}^{x=2n}{e_1^i} \mod e_1$

$e_2^2 + e_2 + 1 \mod e_1 = 1 \mod e_1$

$e_2^2 + e_2 = 0 \mod e_1$

$e_2(e_2 + 1) = 0 \mod e_1$

Below trivial solutions :

- $e_2 = 0 \mod e_1 \implies e_2 = e_1$
- $e_2 + 1 = 0\mod e_1$

First trivial solution $e_2 = e_1$ is not possible solution because $x \gt 2$ so following expression cannot be true:

$$e_1^2 + e_1 + 1 = \sum_{i=0}^{x=2n}{e_1^i}$$

In according to second trivial solution $e_2$ is prime number and greater than $e_1$ (demonstrated above) so $e_2$ must be an odd prime number. 

$$e_2 \mod 2 = 1$$
$$e_2 + 1 \mod 2 = 0$$

$e_2 + 1$ is an even number so $e_1$ must be an even prime number, but there is only a even prime number.

$$e_1=2$$

Applying geometric series expansion

$$1 + e_2 + e_2^2 = 2^{x + 1} - 1$$

We can rearrange this via the quadratic equation to 

$$e_2 = \frac{-1 \pm \sqrt{1 - 4 (2 - 2^{x + 1})}}{2}$$
​​. 

Trying out a few values using bruteforce

In [2]:
from math import sqrt
from Crypto.Util.number import isPrime

e_1 = 2

# Get X even number > 2
for x in range(4, 1000, 2):
    #Calculate delta
    delta = sqrt(1 - 4 * (2-e_1**(x+1)))
    #Check delta is a even number
    if delta.is_integer() and delta % 2 == 1:
        #Calculate solution
        e_2 = (-1 + delta) / 2
        #Check solution is number prime
        if isPrime(int(e_2)):
            print(f"x = {x}")
            print(f"e_2 = {int(e_2)}")
            break

x = 4
e_2 = 5


Found a solution:

$$e_1 = 2$$
$$e_2 = 5$$
$$x = 4$$

## Calculate $r$

In [3]:
r = 2
while True:
	r = r*x
	if r.nbits()>1024 and isPrime(r-1):
		r = r-1
		break
print(f"r = {r}")

pq = n / r
print(f"pq = {int(pq)}")

r = 10407932194664399081925240327364085538615262247266704805319112350403608059673360298012239441732324184842421613954281007791383566248323464908139906605677320762924129509389220345773183349661583550472959420547689811211693677147548478866962501384438260291732348885311160828538416585028255604666224831890918801847068222203140521026698435488732958028878050869736186900714720710555703168729087
pq = 2568831164001216391723085814750381907619776101034426901403174759467391618210605078570848467650083413438680379369387667565087273640661863634022310558607002234271435330811191028237984656767854196453027160211070151460279834812645949789565339922939294335595067709162681222099349511478717351831922625500415797306141129717452330162411937336759353036791591528421432292490462559775251978424981334757118794552450795075101125115807282820724421921908902623542927660680996119491999557146197554613794696485562100729431343922318598205820907542244089495862145680971876904474271076092142071426558551657348106121733039677

## Finding $p$ and $q$

In order to find $p$ and $q$ it is possible use inverse modules $ip$ and $iq$:

$$ip = p^{-1} \mod q$$
$$iq = q^{-1} \mod p$$

Inverse modules are related throught follow expression (see https://math.stackexchange.com/questions/1700673/why-p-cdot-p-1-mod-q-q-cdot-q-1-mod-p-pq-1/1705450#1705450):

$$p \cdot (p^{-1} \mod q) + q \cdot(q^{-1} \mod p) = p \cdot q + 1$$

Solving equation a solution can be find:

In [4]:
p,q = var("p, q")
eq1 = (ip * p + iq * q - pq - 1 == 0)
eq2 = (p * q == pq)
solutions = solve((eq1, eq2), (p, q), solution_dict=True)


Explore solutions in accordance with Rabin cryptosystem

In [5]:
for sol in solutions:
    sol_p = sol[p]
    sol_q = sol[q]
    if Mod(sol_p, 4)  == 3 and Mod(sol_q, 4) == 3:
        print("Found p and q")
        print(f"p = {sol_p}")
        print(f"q = {sol_q}")
        break
        

Found p and q
p = 176063173097741148664104359089145423788477383400361597752492045840821437360249359542374966372723996224499016372218339772485880478959284289348284168670207484887892789136253419014976352598592035148148172192007126605099425808464034613303669376160066151180481522308774136265021552630525582690853372018950936388227
q = 145903945657910781114921726419359191977233277082579021865422389591099289803191315127166764774577189980761712395623760828770375291742226959377572159906234087829278080117807787308273841127656879014728893089772803725445572058758562663067626290742341269050937102820565754218175277167460621754505331956899498894279


## Recover Message

Obtained $p$, $q$ and $r$ then Euler totien can be calculated $\phi(N)$ in multiRSA

$$\phi(N) = (p-1)\cdot (q-1) \cdot (r-1)$$

In [17]:

phi = (sol_p - 1) * (sol_q - 1) * (r - 1)
print(f"phi = {phi}")

phi = 2673622057446548300555857462503172451254797352698537133726876046766082856291279775743105104413581041696524449173299861290982407504014252576012822687338340915932004452447254606132981991406905971191997639700643598476668022554560135926315328539512732862848782308938090802503866468321105064022893786910794623648815486507665825724103407349374645281377274506305523122323462751520209321631221270112623507598589110436243701027805100412952899229408151648778906508855777644947475803541219557602729788841260010330041184543944983852105641871355940445438195882918851635501679755366987767858113043170267429963222448628121745387231936599314065761867361964577410258274563246958816665092878174223241013204915262249394386816362700513148775864128103600179407085014825530167087126549566195305884334170294627330276422002813560796329161432624359118083067062027576665228335913287487525337379433902114230137847649789533677702604282413581284672659568036785073057848762899291063728849877460970473538797083620637326521305

In [15]:
d1 = pow(int(e_1), -1, int(phi))

ZeroDivisionError: Inverse does not exist.

In [14]:
from Crypto.Util.number import long_to_bytes

d2 = pow(int(e_2), -1, int(phi))
m1 = pow(c1, d1, n)
m2 = pow(c2, d2, n)
print(long_to_bytes(int(m1)))
print(long_to_bytes(int(m2)))

ZeroDivisionError: Inverse does not exist.